In [1]:
import numpy as np
import cv2
import imutils
import pytesseract
# to compute angles
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [37]:
img = cv2.imread("test_images/image5.jpg")
cv2.imshow('Cropped',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv2.imshow('Cropped',grey)
cv2.waitKey(0)
cv2.destroyAllWindows()

rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 5))
blackhat = cv2.morphologyEx(grey, cv2.MORPH_BLACKHAT, rectKern) #blackhat morphological operation to reveal dark characters

# next, find regions in the image that are light
squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
light = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, squareKern)
light = cv2.threshold(light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]


In [4]:
# compute the Scharr gradient representation of the blackhat
# image in the x-direction and then scale the result back to
# the range [0, 255]
gradX = cv2.Sobel(blackhat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = 255 * ((gradX - minVal) / (maxVal - minVal))
gradX = gradX.astype("uint8")



In [68]:
# blur the gradient representation, applying a closing
# operation, and threshold the image using Otsu's method
gradX = cv2.GaussianBlur(gradX, (7, 7), 0)
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKern)
thresh = cv2.threshold(gradX, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

In [69]:
# perform a series of erosions and dilations to clean up the
# thresholded image
thresh = cv2.erode(thresh, None, iterations=3)
thresh = cv2.dilate(thresh, None, iterations=3)

In [70]:
# take the bitwise AND between the threshold result and the
# light regions of the image
thresh = cv2.bitwise_and(thresh, thresh, mask=light)
thresh = cv2.dilate(thresh, None, iterations=2)
thresh = cv2.erode(thresh, None, iterations=1)

# cv2.imshow('Cropped',thresh)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [96]:
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
candidates = sorted(cnts, key=cv2.contourArea, reverse=True)[:5] #We’ll only return up to this many sorted license plate candidate contours.

In [102]:
# initialize the license plate contour and ROI
lpCnt = None
roi = None
# loop over the license plate candidate contours
ar_list = []
for c in candidates:
    print(ar)
    # compute the bounding box of the contour and then use
    # the bounding box to derive the aspect ratio
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    ar_list.append(ar)
    
ar = ar_list[1]

4.339622641509434
4.339622641509434
3.306451612903226
1.139344262295082
0.5192307692307693


In [103]:
minAR = 4
maxAR = 6
if ar >= minAR and ar <= maxAR:
# store the license plate contour and extract the
# license plate from the grayscale image and then
# threshold it
    lpCnt = c
    licensePlate = grey[y:y + h, x:x + w]
    roi = cv2.threshold(licensePlate, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [104]:
cv2.imshow('Cropped',licensePlate)
cv2.waitKey(0)
cv2.destroyAllWindows()